# WRF

* [UCAR-WRF](http://www2.mmm.ucar.edu/wrf/users/downloads.html)
* [Public Domain Notice](http://www2.mmm.ucar.edu/wrf/users/public.html)
* [tutorial](http://www2.mmm.ucar.edu/wrf/OnLineTutorial/index.htm)
* [registration](http://www2.mmm.ucar.edu/wrf/users/download/wrf-regist.php)
* [src](http://www2.mmm.ucar.edu/wrf/users/download/get_sources.html)


In [3]:
######################
# from __future__ import print_function
import os, sys; 
# lib = "/usr/local/apps/ecflow/current/lib/python2.7/site-packages/ecflow"
lib = "/usr/local/apps/ecflow/current/lib/python3.5/site-packages/ecflow"
lib = "/usr/local/lib/python3.5/site-packages/ecflow"
sys.path.append(lib)
import ecf; from ecf import (Client, Defs, Suite, Family, Task, Defstatus, Edit, Trigger)
try: x = Edit(test="value")  # Edit is present in recent ecf.py module
except:
    class Edit(Variables): pass
home = os.getenv("HOME") + "/ecflow_server"
user = os.getenv("USER")

WRF="WRFV3.9.1.1.TAR.gz"  # HERE A PYTHON VARIABLE
FROM="http://www2.mmm.ucar.edu/wrf/src/WRFV3.9.1.1.TAR.gz"    

# SUITE
node = Suite("WRF").add(
    Defstatus("suspended"),
    Edit(ECF_HOME=home, ECF_INCLUDE=home + "/include", ECF_FILES=home + "/files",
         ECF_EXTN=".ecg",  # current convention for generated task template extension
         ECF_JOB_CMD="%ECF_JOB% > %ECF_JOBOUT% 2>&1",  # localhost run
         ECF_URL_CMD="firefox %URL%",
         WRF=WRF,  # PYTHON VARIABLES IS TURNED INTO ECFLOW VARIABLE
         FROM=FROM,
         URL="http://lmdz.lmd.jussieu.fr/", ),
    Family("make").add(
        Family("get").add(Task("cmd").add(
            # Defstatus("complete")
            Edit(CMD="WRF=%WRF%; FROM=%FROM%;\n" +  #ECFLOW VARIABLES TURNED INTO SCRIPT (SHELL) VARIABLE
                 "[ ! -d WRFV3 ] && curl -o $WRF.tgz $FROM && tar -xzvf $WRF.tgz",
                 ARGS=""))),
        Family("compile").add(
            Trigger(["get"]),
            Task("cmd").add(
                Edit(CMD="cd WRFV3 && ./configure && make")), 
        ), ),
    Family("main").add(Task("cmd").add(Edit(CMD="echo", ARGS="YOUR PART"))))

# TASK TEMPLATE
with open(home + "/files/cmd.ecg", 'w') as task_template: print("""#!/bin/bash
%include <head.h>
 %CMD:echo% %ARGS:%
%include <tail.h>""", file=task_template)

# DEFS
defs = Defs()
defs.add_suite(node)
path = '/' + node.name()

# CLIENT
client = Client("localhost@%s" % os.getenv("ECF_PORT", 2500))  # PYTHON CLIENT
client.replace(path, defs)  # load/replace the top node (suite)
client.begin_suite(node.name())  # BEGIN suite: UNKNOWN -> QUEUED
client.resume(path) # RESUME suite: SUSPENDED -> create job and submit

In [2]:
os.system("ecflow_ui &")

0